## Module 9: Access control to online and offline feature store using IAM policies


Feature Store helps improve security without affecting team productivity and innovation. Feature Store allows you to grant or deny access to individuals at the feature group-level and enables cross-account access to Feature Store. For example, you can set up developer accounts to access the offline store for model training and exploration that do not have write access to production accounts. You can set up production accounts to access both online and offline stores.  Similarly, you can setup Data Scientists that are consumers from one Business Unit with read access but Data Scientists from another Business Unit are a producer with write access.

![Granular Access using IAM Policy](../../images/fs_security_iam_policy_overview.png "Granular Access using IAM Policy")

### IAM Roles

You can use AWS Identity and Access Management (https://aws.amazon.com/iam/) (IAM) roles to give or restrict granular access to specific features for specific users or groups. This is applicable for both the online and offline feature store. Access control is enabled through both the API and AWS KMS key access.  For more information about using AWS KMS permissions for feature store access control, please refer to this document (https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-security.html).

#### Imports 

In [6]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role
import sagemaker
import logging
import boto3
import pandas as pd
import time
import re
import os
import sys
import json
import pandas as pd
sys.path.append('../..')
from utilities import Utils

In [7]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Initialize default parameters

In [8]:
sagemaker_session = sagemaker.Session()
account_id = sagemaker_session.account_id()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()
s3_client = boto3.client('s3', region_name=region)
query_results= 'sagemaker-featurestore-workshop'
prefix = 'sagemaker-feature-store'

#### Initialize boto3 runtime

In [9]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = sagemaker.Session(boto_session=boto_session, 
                                          sagemaker_client=sagemaker_client, 
                                          sagemaker_featurestore_runtime_client=featurestore_runtime)

#### Retrieve the orders products and customers feature group names

In this notebook, we will be using the feature groups created in Module 1 of the workshop. In case you would like to use a different feature group, please uncomment and replace **ORDERS_FEATURE_GROUP_NAME**, **CUSTOMERS_FEATURE_GROUP_NAME**, **PRODUCTS_FEATURE_GROUP_NAME** with the name of your Feature Groups for example. 

In [10]:
# Retrieve FG names (when running previous modules)
#%store -r customers_feature_group_name
#%store -r orders_feature_group_name
#%store -r products_feature_group_name

orders_feature_group_name = 'fscw-orders-02-13-20-20'
customers_feature_group_name = 'fscw-customers-02-13-20-20'
products_feature_group_name = 'fscw-products-02-13-20-20'

logger.info(f'Customers feature group name = {customers_feature_group_name}')
logger.info(f'Products feature group name = {products_feature_group_name}')
logger.info(f'Orders feature group name = {orders_feature_group_name}')

customers_fg = FeatureGroup(name=customers_feature_group_name, sagemaker_session=feature_store_session)  
orders_fg = FeatureGroup(name=orders_feature_group_name, sagemaker_session=feature_store_session)
products_fg = FeatureGroup(name=products_feature_group_name, sagemaker_session=feature_store_session)

Customers feature group name = fscw-customers-02-13-20-20
Products feature group name = fscw-products-02-13-20-20
Orders feature group name = fscw-orders-02-13-20-20


### IAM policy update
Add the below policy to the IAM role (SageMaker Execution Role) used for this notebook as inline policy. These will allow you to modify the IAM policy programatically in the following sections.
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "iam:CreatePolicy",
                "iam:DetachRolePolicy",
                "iam:ListAttachedRolePolicies",
                "iam:DeletePolicy",
                "iam:AttachRolePolicy"
            ],
            "Resource": "*"
        }
    ]
}
```

In [11]:
sagemaker_client.describe_feature_metadata(
    FeatureGroupName='fscw-orders-02-13-20-20',
    FeatureName="order_id"
)

AttributeError: 'SageMaker' object has no attribute 'describe_feature_metadata'

### Add tags to feature groups
Tags are key value pairs and in this section you will create and add a tag to feature groups: *team=A* for *customers* feature group and *team=B* for *orders* feature group.

In [7]:
# Utility function to add tag to a feature group
# If the same tag exists it overides previous value
def feature_add_tag(fg_name, tag, value):

    fg = FeatureGroup(name=fg_name, sagemaker_session=feature_store_session)    
    fg_arn = fg.describe()['FeatureGroupArn']
    
    # Add the DataBrew Profile URL tag to FG
    response = sagemaker_client.add_tags(
        ResourceArn= fg_arn,
        Tags=[
            {
                'Key': tag,
                'Value': Utils._escape_tag_chars(value)
            },
        ]
    )
    
    return response

# Use utility function to add tags to feature groups
fg_team_tag = 'team'

# Add tag "team" and value "A" for customers, and "B" for orders
feature_add_tag(customers_feature_group_name, fg_team_tag, "A")
feature_add_tag(orders_feature_group_name, fg_team_tag, "B")
    

{'Tags': [{'Key': 'team', 'Value': 'B'}],
 'ResponseMetadata': {'RequestId': '479e451b-3eaa-45da-96bb-375714bb64d9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '479e451b-3eaa-45da-96bb-375714bb64d9',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '37',
   'date': 'Tue, 17 May 2022 15:15:10 GMT'},
  'RetryAttempts': 0}}

### Create and add IAM policies based on tags to the existing role

Define IAM policies for offline feature store access and online feature store respectively

In [8]:
"""creation of iam policy for offline feature store."""
    
_iam_access_policy = {
    "Version": "2012-10-17",
    "Statement": [
    {
        "Sid": "VisualEditor0",
        "Effect": "Deny",
        "Action": [
            "sagemaker:DeleteFeatureGroup",
            "sagemaker:DescribeFeatureGroup",
            "sagemaker:CreateFeatureGroup",
            "sagemaker:GetRecord",
        ],
        "Resource": "*",
        "Condition": {
            "ForAnyValue:StringEquals": {
                "aws:ResourceTag/team":"A"
            }
        }
    }
    ]
}


Create IAM policies and attach them to the IAM role (SageMaker Execution Role)

In [9]:
# Attach IAM policy to restrict SageMaker Execution Role
timestamp = int(time.time())
iam_client = boto3.client('iam')
role_name = role.split('/')[-1] # get the role name from role arn

policy_res = iam_client.create_policy(
    PolicyName=f'Amazon_SageMaker_Granular_Policy_{timestamp}',
    PolicyDocument=json.dumps(_iam_access_policy)
)
policy_arn = policy_res['Policy']['Arn']

policy_attach_res = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

### 1. Test Deny with one tag and offline feature store

**Open Feature Store and click on Feature group with tag team = "A"**, and the blow page show the describe api call failed due to the IAM policy.
![access_deny](../../images/fs_security_access_deny.png) 

But when opening feature group with tag team = "B", the describe api call was successful

![allow_access_team_b](../../images/fs_security_allow_access_team_b.png) 

### 2. Test Deny with API for online feature store

In this test, you are calling the GetRecord API to retrieve a feature group record from customers feature group. It will fail because the SageMaker user does not have access to feature groups that are tagged with team "A".

In [14]:
customer_id = 'C7469'
feature_record = featurestore_runtime.get_record(FeatureGroupName=customers_feature_group_name, RecordIdentifierValueAsString=str(customer_id))
print(json.dumps(feature_record["Record"], indent=2))

ClientError: An error occurred (AccessDeniedException) when calling the GetRecord operation: User: arn:aws:sts::227246955871:assumed-role/AmazonSageMaker-ExecutionRole-20220214T210396/SageMaker is not authorized to perform: sagemaker:GetRecord on resource: arn:aws:sagemaker:us-east-1:227246955871:feature-group/fscw-customers-02-13-20-20 with an explicit deny in an identity-based policy

In this test, you are calling the GetRecord API to retrieve a feature group record from orders feature group. It will be successful because the SageMaker user has been given access to feature groups except the ones that are tagged with team "A".

In [11]:
record_id = 'O2235'
feature_record = featurestore_runtime.get_record(FeatureGroupName=orders_feature_group_name, RecordIdentifierValueAsString=str(record_id))
print(json.dumps(feature_record["Record"], indent=2))

[
  {
    "FeatureName": "order_id",
    "ValueAsString": "O2235"
  },
  {
    "FeatureName": "customer_id",
    "ValueAsString": "C7469"
  },
  {
    "FeatureName": "product_id",
    "ValueAsString": "P6636"
  },
  {
    "FeatureName": "purchase_amount",
    "ValueAsString": "0.2853465346534653"
  },
  {
    "FeatureName": "is_reordered",
    "ValueAsString": "0"
  },
  {
    "FeatureName": "event_time",
    "ValueAsString": "2022-02-13T20:18:06.768Z"
  },
  {
    "FeatureName": "n_days_since_last_purchase",
    "ValueAsString": "0.0678294573643411"
  }
]


### Clean up

First delete the tag based granular access IAM policy 

In [15]:
# Detach iam policy added to the SageMaker execution role
policy_detach_res = iam_client.detach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

# Delete the IAM policy
delete_policy_res = iam_client.delete_policy(
    PolicyArn=policy_arn
)


Finally delete the inline policy you added to the IAM role (SageMaker Execution Role) earlier in this notebook.
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "iam:CreatePolicy",
                "iam:DetachRolePolicy",
                "iam:ListAttachedRolePolicies",
                "iam:DeletePolicy",
                "iam:AttachRolePolicy"
            ],
            "Resource": "*"
        }
    ]
}
```